In [29]:
import gc
import pandas as pd

In [30]:
gc.collect() # removes unused/unneeded objects
trueNews = pd.read_csv('newsData/True.csv')
fakeNews = pd.read_csv('newsData/Fake.csv')

print("Preview of Fake News Articles:")
display(fakeNews.head())
print("Preview of True News Articles:")
display(trueNews.head())

Preview of Fake News Articles:


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


Preview of True News Articles:


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [31]:
print(f"True News Dataset Shape: {trueNews.shape}")
print(f"Fake News Dataset Shape {fakeNews.shape}")

print("\nColumns in the Dataset:")
print(trueNews.columns)

print("\nMissing Values in True News:")
print(trueNews.isnull().sum().sum())

print("\nMissing Values in Fake News:")
print(fakeNews.isnull().sum().sum())


True News Dataset Shape: (21417, 4)
Fake News Dataset Shape (23481, 4)

Columns in the Dataset:
Index(['title', 'text', 'subject', 'date'], dtype='object')

Missing Values in True News:
0

Missing Values in Fake News:
0


In [32]:
print(trueNews.info())
print("\n", trueNews['subject'].value_counts())
print("\n", trueNews.nunique())
print("\n", f"Duplicated rows: {trueNews.duplicated().sum()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
None

 subject
politicsNews    11272
worldnews       10145
Name: count, dtype: int64

 title      20826
text       21192
subject        2
date         716
dtype: int64

 Duplicated rows: 206


In [33]:
print(fakeNews.info())
print("\n", fakeNews['subject'].value_counts())
print("\n", fakeNews.nunique())
print("\n", f"Duplicated rows: {fakeNews.duplicated().sum()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
None

 subject
News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: count, dtype: int64

 title      17903
text       17455
subject        6
date        1681
dtype: int64

 Duplicated rows: 3


In [ ]:
fakeNews = fakeNews[fakeNews['subject'].isin(['politics', 'News'])].copy() # filters data
subject_mapping = { # maps news subjects
    'politics': 'politicsNews',
    'News': 'worldnews'
}
fakeNews['subject'] = fakeNews['subject'].map(subject_mapping)

print("Subjects in fakeNews after cleaning:", fakeNews['subject'].unique())
print(f"Cleaned Fake News shape: {fakeNews.shape}")
print("Subjects i trueNews (already chosen):", trueNews['subject'].unique())

Subjects in fakeNews after cleaning: ['worldnews' 'politicsNews']
Cleaned Fake News shape: (15891, 4)
Subjects i trueNews (already chosen): ['politicsNews' 'worldnews']


In [ ]:
# confirming consistency after cleaning

print("Subjects in trueNews:", trueNews['subject'].unique())
print("Subjects in fakeNews:", fakeNews['subject'].unique())

print("\nMissing subjects in trueNews:", trueNews['subject'].isnull().sum())
print("Missing subjects in fakeNews:", fakeNews['subject'].isnull().sum())

print(f"\nTrue News shape: {trueNews.shape}")
print(f"Fake News shape:', {fakeNews.shape}")

Subjects in trueNews: ['politicsNews' 'worldnews']
Subjects in fakeNews: ['worldnews' 'politicsNews']

Missing subjects in trueNews: 0
Missing subjects in fakeNews: 0

True News shape: (21417, 4)
Fake News shape:', (15891, 4)


note to self; for most deep learning cases, you'll want to use one-hot encoding 
* this is because it helps to avoid overfit and bias in the learning process
* the computer might assume label encoding is a stand-in for rankings

however, one-hot encoding would be considered overkill in this situation 
* specifically, it can increase memory usage
* so, instead i'll use label encoding (as 0s and 1s) for the subjects

In [41]:
# 1 = real news
# 0 = fake news 

fakeNews['labelAuthenticity'] = 0
trueNews['labelAuthenticity'] = 1

# 1 = worldnews
# 0 for politicsNews

subject_mapping = {
    'politicsNews': 0,
    'worldnews': 1
}

trueNews['labelSubject'] = trueNews['subject'].map(subject_mapping)
fakeNews['labelSubject'] = fakeNews['subject'].map(subject_mapping)

newsDF = pd.concat([trueNews, fakeNews], ignore_index = True)
print("Combined Dataset Shape:", newsDF.shape)
display(newsDF.head())

Combined Dataset Shape: (37308, 6)


,title,text,subject,date,labelAuthenticity,labelSubject
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,0
